# Without Pre-trained weights

In [129]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

<img src="optimizer_graph.png" width="200px">

## Reading Data

We are reading MNIST data from http://yann.lecun.com/exdb/mnist/


In [130]:
from tensorflow.examples.tutorials.mnist import input_data
fashion_mnist = input_data.read_data_sets('data/fashion',one_hot=True)

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [132]:
print(fashion_mnist.train.num_examples) # Number of training data
print(fashion_mnist.test.num_examples) # Number of test data

55000
10000


## Setting hyper-parameters

In [133]:
# architecture hyper-parameter
learning_rate = 0.01
training_iters = 100000
batch_size = 128
display_step = 20

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

## Defining x and y as placeholders

In [134]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(x.shape, y.shape)

(?, 784) (?, 10)


## Defining model using conv2d, Relu and Maxpooling

In [135]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [136]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [137]:
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Initializing Weights and Biases

In [138]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Define cost, optimizer and accuracy

In [139]:
# Create the model
model = conv_net(x, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Tensor("Add_17:0", shape=(?, 10), dtype=float32)


## Running the session 
This will run the graph and use all the tensors that were previously defined

In [141]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = fashion_mnist.train.next_batch(batch_size)   
        #print(batch_x.shape, batch_y.shape)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Loss= " + \
                  "{:.3f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        step += 1
    
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: fashion_mnist.test.images[:256],
                                      y: fashion_mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 2560, Loss= 4105.302, Training Accuracy= 0.625
Iter 5120, Loss= 864.092, Training Accuracy= 0.711
Iter 7680, Loss= 413.374, Training Accuracy= 0.789
Iter 10240, Loss= 542.875, Training Accuracy= 0.766
Iter 12800, Loss= 378.996, Training Accuracy= 0.781
Iter 15360, Loss= 294.518, Training Accuracy= 0.805
Iter 17920, Loss= 217.109, Training Accuracy= 0.836
Iter 20480, Loss= 274.553, Training Accuracy= 0.773
Iter 23040, Loss= 114.387, Training Accuracy= 0.844
Iter 25600, Loss= 159.929, Training Accuracy= 0.789
Iter 28160, Loss= 98.895, Training Accuracy= 0.852
Iter 30720, Loss= 197.119, Training Accuracy= 0.742
Iter 33280, Loss= 120.796, Training Accuracy= 0.773
Iter 35840, Loss= 74.064, Training Accuracy= 0.883
Iter 38400, Loss= 67.064, Training Accuracy= 0.820
Iter 40960, Loss= 116.482, Training Accuracy= 0.758
Iter 43520, Loss= 96.857, Training Accuracy= 0.711
Iter 46080, Loss= 82.009, Training Accuracy= 0.766
Iter 48640, Loss= 48.711, Training Accuracy= 0.789
Iter 51200, Loss= 42